In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense , Conv2D, Dropout, Flatten, MaxPooling2D
from keras.utils import np_utils
from keras import regularizers
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import h5py
import os
import cv2
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
#we always initialize the random number generator to a constant seed #value for reproducibility of results.
seed = 7
np.random.seed(seed)

In [3]:
# load data from the path specified by the user
def data_loader(path_train,path_test):
   train_list=os.listdir(path_train)  
   '''
   # Map class names to integer labels
   train_class_labels = { label: index for index, label in enumerate(class_names) } 
   '''
   # Number of classes in the dataset
   num_classes=len(train_list)
   
   # Empty lists for loading training and testing data images as well as corresponding labels
   x_train=[]
   y_train=[]
   x_test=[]
   y_test=[]
   
   # Loading training data
   for label,elem in enumerate(train_list):
           
           path1=path_train+'/'+str(elem)
           images=os.listdir(path1)
           for elem2 in images:
               path2=path1+'/'+str(elem2)
               # Read the image form the directory
               img = cv2.imread(path2)   
               # Append image to the train data list
               x_train.append(img)
               # Append class-label corresponding to the image
               y_train.append(str(label))
   
           # Loading testing data
           path1=path_test+'/'+str(elem)
           images=os.listdir(path1)
           for elem2 in images:
               path2=path1+'/'+str(elem2)
               # Read the image form the directory
               img = cv2.imread(path2)
               # Append image to the test data list
               x_test.append(img)
               # Append class-label corresponding to the image
               y_test.append(str(label))
           
   # Convert lists into numpy arrays
   x_train=np.asarray(x_train)
   y_train=np.asarray(y_train)
   x_test=np.asarray(x_test)
   y_test=np.asarray(y_test)
   return x_train,y_train,x_test,y_test

In [4]:
path_train='./Data/train'
path_test='./Data/test'


In [5]:
X_train,y_train,X_test,y_test=data_loader(path_train,path_test)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28, 3)
(60000,)
(10000, 28, 28, 3)
(10000,)


In [6]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

In [7]:
# forcing the precision of the pixel values to be 32 bit
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [8]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255.
X_test = X_test / 255.

In [9]:
# one hot encode outputs using np_utils.to_categorical inbuilt function
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [10]:
#Splitting the trining data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
# define baseline model
#The model is a simple neural network with one hidden layer with the same number of neurons as there are inputs (784)
def baseline_model():
	# create model
	model = Sequential()
	#We will add 2 Convolution layers with 32 filters of 3x3, keeping the padding as same
	model.add(Conv2D(32, (3, 3), strides=(1, 1), padding = 'same' , input_shape = input_shape, activation = 'relu', kernel_initializer = 'glorot_uniform', kernel_regularizer = regularizers.l2(0.01)))
	model.add(Conv2D(32, (3, 3), strides=(1, 1), padding = 'same', activation = 'relu', kernel_initializer = 'glorot_uniform', kernel_regularizer = regularizers.l2(0.01)))
	#Pooling the feature map using a 2x2 pool filter
	model.add(MaxPooling2D((2, 2), strides=(2, 2), padding = 'valid'))
	#Adding 2 more Convolutional layers having 64 filters of 3x3
	model.add(Conv2D(64, (3, 3), strides=(1, 1), padding = 'same', activation = 'relu', kernel_initializer = 'glorot_uniform', kernel_regularizer = regularizers.l2(0.01)))
	model.add(Conv2D(64, (3, 3), strides=(1, 1), padding = 'same', activation = 'relu', kernel_initializer = 'glorot_uniform', kernel_regularizer = regularizers.l2(0.01)))
	#Flatten the feature map
	model.add(Flatten())
	#Adding FC Layers
	model.add(Dense(500, activation='relu'))
	model.add(Dropout(0.3))
	model.add(Dense(100, activation='relu'))
	model.add(Dropout(0.3))
	#A softmax activation function is used on the output
	#to turn the outputs into probability-like values and 
	#allow one class of the 10 to be selected as the model's output #prediction.
	model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
	#Checking the model summary
	model.summary()
	# Loading weigths
	#model.load_weights('./CNN.h5')
	# Compile model
	sgd = SGD(lr = 0.001, momentum = 0.0005, decay = 0.0005)
	adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0005)
	model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
	#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
	return model


In [12]:
# build the model
model = baseline_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               6272500   
__________

In [14]:
# Fit the model
#The model is fit over 10 epochs with updates every 200 images. The test data is used as the validation dataset
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, batch_size=200, verbose=2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/1
 - 139s - loss: 0.2271 - acc: 0.9655 - val_loss: 0.1380 - val_acc: 0.9811


In [15]:
# checkpoint
filepath='./CNN.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [16]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=200, callbacks=callbacks_list, verbose=2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
 - 140s - loss: 0.1429 - acc: 0.9760 - val_loss: 0.1068 - val_acc: 0.9848

Epoch 00001: val_acc improved from -inf to 0.98475, saving model to ./CNN.h5
Epoch 2/2
 - 138s - loss: 0.1155 - acc: 0.9797 - val_loss: 0.1008 - val_acc: 0.9828

Epoch 00002: val_acc did not improve from 0.98475


In [17]:
#Saving the model
model.save_weights('./CNN.h5')

In [18]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))


Baseline Error: 1.61%
